## Importando Bibliotecas

In [1]:
import random

## Funções - Simple Tabulation Hash

In [2]:
def particionar(x):
    x = format(x, '064b')
    particoes = []
    for i in range(0, len(x), 8):
        particoes.append(x[i:i+8])
    return particoes

In [3]:
def tabelaAleatoria():
    tabelas = []
    random.seed(10)
    for i in range(0, 8):
        tabela = []
        for j in range(0, 256):
            tabela.append(random.randint(0, 2**64))
        tabelas.append(tabela)
    return tabelas

In [4]:
def funcaoHash(x):
    particoesBinarias = particionar(x)
    decimais = []
    for i in range(0, 8):
        decimais.append(int(particoesBinarias[i], 2))
    tabelas = tabelaAleatoria()
    h = tabelas[0][decimais[0]] ^ tabelas[1][decimais[1]]
    for i in range(2, 8):
        h = h ^ tabelas[i][decimais[i]]
    return h

## Table Doubling e Halving

In [5]:
def inicilizarTabela(m):
    tabela = []
    for i in range(0, m):
        tabela.append(None)
    return tabela

In [6]:
def limpar(tabela):
    global contadorTag
    contadorTag = 0
    novaTabela = inicilizarTabela(len(tabela))
    novaTabela = remapear(tabela, novaTabela, False)
    return novaTabela

In [7]:
def remapear(tabelaAntiga, tabela, adicionarLog):
    global contadorValores
    contadorValores = 0
    for valor in tabelaAntiga:
        if(valor != None and valor !='#'):
            tabela = incluir(valor, tabela, adicionarLog)
    return tabela

In [8]:
def doubling(tabela, adicionarLog):
    novaTabela = inicilizarTabela(len(tabela) * 2)
    novaTabela = remapear(tabela, novaTabela, adicionarLog)
    return novaTabela

In [9]:
def halving(tabela, adicionarLog):
    epsilon = 1
    if(len(tabela)/2 >= (1+epsilon)):
        novaTabela = inicilizarTabela(int(len(tabela)/2))
        novaTabela = remapear(tabela, novaTabela, adicionarLog)
    return novaTabela

## Funções - Tabela Hash

In [10]:
def incluir(valor, tabela, adicionarLog):
    global contadorValores
    hX = funcaoHash(valor)
    valorInserido = False
    houveDoubling = False
    i = 0
    while(not valorInserido):
        if(i < len(tabela)):
            posicao = (hX + i) % len(tabela)
            if(tabela[posicao] == None or tabela[posicao] == '#'):
                tabela[posicao] = valor
                contadorValores = contadorValores + 1
                valorInserido = True
                if(adicionarLog):
                    logs.append('INC:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
                    print('INC:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
            else:
                i = i + 1
        else:
            tabela = doubling(tabela, False)
            houveDoubling = True
            i = 0
    if(houveDoubling):
        logs.append('DOBRAR TAM:'+str(len(tabela))+'\n\n')
        print('DOBRAR TAM:'+str(len(tabela))+'\n\n')
    return tabela

In [11]:
def buscar(valor, tabela, adicionarLog):
    hX = funcaoHash(valor)
    valorEncontrado = False
    i = 0
    while(not valorEncontrado and i<len(tabela)):
        posicao = (hX + i) % len(tabela)
        if(tabela[posicao] == valor):
            if(adicionarLog):
                logs.append('BUS:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
                print('BUS:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
            return posicao
        else:
            i = i + 1
    posicao = -1
    if(adicionarLog):
        logs.append('BUS:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
        print('BUS:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
    return -1

In [12]:
def remover(valor, tabela):
    global contadorValores
    global contadorTag
    hX = funcaoHash(valor)
    posicao = buscar(valor, tabela, False)
    houveHalving = False
    if(posicao != -1):
        tabela[posicao] = '#'
        contadorValores = contadorValores - 1
        contadorTag = contadorTag + 1
        logs.append('REM:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
        print('REM:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
        if(contadorTag >= len(tabela)/4):
            tabela = limpar(tabela)
        if(contadorValores <= len(tabela)/4):
            tabela = halving(tabela, False)
            houveHalving = True
    else:
        logs.append('REM:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
        print('REM:'+str(valor)+'\n'+str(hX % len(tabela))+' '+str(posicao)+'\n\n')
    if(houveHalving):
        logs.append('METADE TAM:'+str(len(tabela))+'\n\n')
        print('METADE TAM:'+str(len(tabela))+'\n\n')
    return tabela

## Execução

In [13]:
logs = []
contadorTag = 0
contadorValores = 0

In [14]:
def executar():
    tabela = inicilizarTabela(4)
    logs.append('TAM:'+str(len(tabela))+'\n\n')
    entrada = open('entrada.txt','r')
    for comando in entrada:
        com, valor = comando.split(':')
        if(com == 'INC'):
            tabela = incluir(int(valor), tabela, True)
        elif(com == 'BUS'):
            buscar(int(valor), tabela, True)
        elif(com == 'REM'):
            tabela = remover(int(valor), tabela)
    entrada.close()
    saida = open('saida.txt', 'w')
    saida.writelines(logs)
    saida.close()

In [15]:
executar()

INC:10
3 3


INC:20
1 1


REM:10
3 3


METADE TAM:2


INC:15
0 0


INC:15
0 2


DOBRAR TAM:4


REM:17
0 -1


BUS:42
3 -1


INC:42
3 3


INC:43
0 0


DOBRAR TAM:8


